In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
import cv2

In [17]:
pokemon_embeddings = pd.read_csv('/kaggle/input/material/pokemon_species_embeddings.csv')

In [18]:
#Exclude the last column (labels)
pokemon_embeddings_values = pokemon_embeddings.iloc[:, :-1].values 

In [19]:
# Label Column
pokemon_species_labels = pokemon_embeddings.iloc[:, -1].values

In [20]:
# Left this to recognize the error 

# import tensorflow as tf
# from keras.models import load_model

# face_embedding_model = load_model('/kaggle/input/material/faceEmbedings.h5')

In [21]:
# not simply loaded the model as Keras cannot locate the 'mse' function during deserialization. 
# This problem may occur if the custom loss function is not properly registered for serialization.
# This c=issue can be resolved by defining and registering mse here only

import keras.backend as K
import tensorflow as tf
from keras.models import load_model

def mse(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred), axis=-1)

from keras.utils import custom_object_scope

with custom_object_scope({'mse': mse}):
    face_embedding_model = load_model('/kaggle/input/material/faceEmbedings.h5')

In [22]:
# used to extract face embedings of user
def fun(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    # Extract the face embedding using your pre-trained model
    face_embedding = face_embedding_model.predict(image[np.newaxis, ...])
    return face_embedding

In [23]:
user_embedding = fun('/kaggle/input/images/WIN_20211119_12_01_47_Pro (2).jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step


In [27]:
# euclid or cosine cant take 3D input so change them into 3D only
user_embedding_reshaped = np.tile(user_embedding, (pokemon_embeddings_values.shape[0], 1))

In [28]:
distances = euclidean_distances(user_embedding_reshaped, pokemon_embeddings_values)

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 128 while Y.shape[1] == 25088